In [9]:
#essentials
import numpy as np
import pandas as pd

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import shap
import ast

#tools/metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

#modeling
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn import linear_model
from sklearn.linear_model import Ridge


In [10]:
kp =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\KenPom Team Metrics.csv")
kp_conf_rat =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\KP_Conf_Ratings.csv")

df =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\training_data_to_use.csv")
models_parameters =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\model_parameters_defense.csv")

model_hyperparameters = pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\model_hyper_parameters_defense.csv")

In [11]:
df.head(3)

,Unnamed: 0,player_type,Team,Team_n1,Unnamed..0_x,Player.Name,season,conf_abbr,games,games_started,...,Calc B,Calc B_n1,Calc C,Calc C_n1,Calc Overall A,Calc Overall A2,Calc Overall A3,Calc Overall A_n1,Calc Overall A2_n1,Calc Overall A3_n1
0,1,1,JACKSONVILLE,JACKSONVILLE,16199.0,aamahne-santos-1,2018-19,A-Sun,32,31,...,513.80,618.80,-800.06,-516.80,594.54,168.82,7450.10,720.80,360.40,6834.0
1,2,1,CLEMSON,CLEMSON,7084.0,aamir-simms-1,2018-19,ACC,34,34,...,1100.55,1676.10,493.58,1155.15,1280.64,1374.02,8037.35,1985.65,2272.55,9769.7
2,3,1,CLEMSON,CLEMSON,7085.0,aamir-simms-1,2019-20,ACC,30,30,...,1676.10,1453.86,1155.15,1461.24,1985.65,2272.55,9769.70,1719.54,2095.92,9778.5


In [12]:
models_parameters.head(3)

,regression_model,parameter_list,no_of_features
0,linear_regression,"['player_type','games', 'games_started', 'mp_p...",4
1,ridge_regression,"['player_type', 'games', 'fg_per_g', 'fga_per_...",4
2,xg_boost,"['player_type', 'trb_per_min', 'trb_per_poss',...",4


In [13]:
model_hyperparameters.head(3)

,model,best_score,best_params
0,linear_regression,0.630944,{}
1,ridge_regression,0.634638,{'alpha': 150}
2,xg_boost,0.616541,"{'learning_rate': 0.1, 'max_depth': 4, 'min_ch..."


In [15]:
def r2(actual: np.ndarray, predicted: np.ndarray):
    """ R2 Score """
    return r2_score(actual, predicted)

def adjr2(actual: np.ndarray, predicted: np.ndarray, rowcount: int, featurecount: int):
    """ Adjusted R2 Score """
    return 1-(1-r2(actual,predicted))*(rowcount-1)/(rowcount-featurecount)

In [29]:
columns = models_parameters['parameter_list'][2]
columns = ast.literal_eval(columns)
print(len(columns))

92


In [32]:
hyperparameters = model_hyperparameters['best_params'][2]
print(hyperparameters)

{'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 0.1, 'subsample': 0.5}


In [33]:
master_table  = df[columns]
print(master_table.shape)
    
y= (df['defence_def_rtg']).to_frame()
print(y.shape)

(7747, 92)
(7747, 1)


In [34]:
# split the data with 90% in training set
X_train, X_test, y_train, y_test = train_test_split(master_table, y, random_state=0,
                                    train_size=0.85)

In [35]:
model_xg = XGBRegressor(hyperparameters)
model_lr.fit(X_train, y_train)
y_predicted = model_lr.predict(X_test)
print(y_predicted.shape)

(1163, 1)


D:\Anaconda\lib\site-packages\xgboost\core.py:727: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


In [25]:
'''
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_predicted = model_lr.predict(X_test)
print(y_predicted.shape)
'''

(1163, 1)


In [36]:
y_predicted.shape[0]

1163

In [37]:
no_of_features = len(columns)
no_of_rows = y_predicted.shape[0]

print(adjr2(y_test,y_predicted,no_of_rows, no_of_features ))
print(r2(y_test,y_predicted))

0.6293656082657113
0.6583911931605653
